In [1]:
%env AWS_DEFAULT_PROFILE=protecso

env: AWS_DEFAULT_PROFILE=protecso


In [2]:
import boto3
import dynamodbgeo
import uuid
dynamodb = boto3.client('dynamodb', region_name='us-east-1')

In [3]:
config = dynamodbgeo.GeoDataManagerConfiguration(dynamodb, 'pest_reports')
geoDataManager = dynamodbgeo.GeoDataManager(config)

In [ ]:
# Pick a hashKeyLength appropriate to your usage
config.hashKeyLength = 5

# Use GeoTableUtil to help construct a CreateTableInput.
table_util = dynamodbgeo.GeoTableUtil(config)
create_table_input=table_util.getCreateTableRequest()

#tweaking the base table parameters as a dict
create_table_input["ProvisionedThroughput"]['ReadCapacityUnits']=5

# Use GeoTableUtil to create the table
table_util.create_table(create_table_input)

Creating table [pest_reports]
Waiting for table [pest_reports] to be created
Table created


In [ ]:
# this is an example of how to put an item in the table
PutItemInput = {
        'Item': {
            'Country': {'S': "Peru"},
            'Capital': {'S': "Lima"},
            'year': {'S': '1000'}
        },
        'ConditionExpression': "attribute_not_exists(hashKey)" # ... Anything else to pass through to `putItem`, eg ConditionExpression

}
geoDataManager.put_Point(dynamodbgeo.PutPointInput(
        dynamodbgeo.GeoPoint(36.879163, 10.243120), # latitude then latitude longitude
         str( uuid.uuid4()), # Use this to ensure uniqueness of the hash/range pairs.
         PutItemInput # pass the dict here
        ))

{'ResponseMetadata': {'RequestId': 'G3N4753TSSO43L8DT3OA4J064FVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 24 Nov 2024 03:36:17 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'G3N4753TSSO43L8DT3OA4J064FVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [11]:
# i want to generate synthetic inside some 4 vertex regions with this coordinates:
regions = [
	{
		'name': 'Acolla',
		'vertexA': [-11.690196, -75.589006],
		'vertexB': [-11.684839, -75.562477],
		'vertexC': [-11.690196, -75.589006],
		'vertexD': [-11.690196, -75.589006]
	},
	{
		'name': 'Matahuasi',
		'vertexA': [-11.801587, -75.502701],
		'vertexB': [-11.935018, -75.356133],
		'vertexC': [-11.876565, -75.275109],
		'vertexD': [-11.770377, -75.397332]
	},
	]


# any input point generated should have the following structure
"""
The | separator means that the data must take a value from the values separated by |
input_data =
{
	Item': {
            'country': {'S': "Peru"},
            'region': {'S': "Acolla | Matahuasi"},
            'date': {'S': 'iso date (day) string ranging from 2024-11-20 to 2024-11-24'},
						'pest': {'S': 'Podredumbre blanca | Moho gris | Mosca minadora | Polilla de calabaza | Araña roja'}
						'description': {'S': '
        },
        'ConditionExpression': "attribute_not_exists(hashKey)" 
}

if Acolla or if Matahuasi then generate generated_point = dynamodbgeo.GeoPoint(lat,lon)

then put point in the table
geoDataManager.put_Point(dynamodbgeo.PutPointInput(
        generated_point, # latitude then latitude longitude
         str( uuid.uuid4()), # Use this to ensure uniqueness of the hash/range pairs.
         input_item # pass the dict here
        ))
"""

pests = { # match a pest with a litle description including 1) What crops affects (max 5) 2) what it does 3) how to act against it
	'Podredumbre blanca' : 'Afecta a la lechuga, tomate, pimiento, pepino y calabaza. Causa la descomposición de los tejidos de la planta. Usar fungicidas',
	'Moho gris' : 'Afecta a la uva, fresa, tomate, manzana y pera. Causa la descomposición de los tejidos de la planta. Usar fungicidas',
	'Mosca minadora' : 'Afecta a la papa, tomate, pimiento, pepino y calabaza. Las larvas se alimentan de los tejidos de la planta. Usar insecticidas',
	'Polilla de calabaza' : 'Afecta a la calabaza, pepino, pimiento, tomate y sandía. Las larvas se alimentan de los tejidos de la planta. Usar insecticidas',
	'Araña roja' : 'Afecta a la papa, tomate, pimiento, pepino y calabaza. Las larvas se alimentan de los tejidos de la planta. Usar insecticidas'
}

# generate 500 points and their input data associated
import random
import datetime
data = [] # [(input_data, generated_point)]
for i in range(500):
	# generate a point in the region
	region = regions[i%2]
	# lat and lon should be inside the 4 vertex. Maybe you can generate the points in a radius from the center of the region
	# first calculate the center of the 4 points. Then calculate an apropiate radius
	center_lat = (region['vertexA'][0] + region['vertexB'][0] + region['vertexC'][0] + region['vertexD'][0])/4
	center_lon = (region['vertexA'][1] + region['vertexB'][1] + region['vertexC'][1] + region['vertexD'][1])/4
	# calculate the radius as the the min distance from any 2 vertex
	radius = min(
		((region['vertexA'][0] - region['vertexB'][0])**2 + (region['vertexA'][1] - region['vertexB'][1])**2)**0.5,
		((region['vertexB'][0] - region['vertexC'][0])**2 + (region['vertexB'][1] - region['vertexC'][1])**2)**0.5,
		((region['vertexC'][0] - region['vertexD'][0])**2 + (region['vertexC'][1] - region['vertexD'][1])**2)**0.5,
		((region['vertexD'][0] - region['vertexA'][0])**2 + (region['vertexD'][1] - region['vertexA'][1])**2)**0.5
	)
	corrected_radius = radius/2
	lat = center_lat + random.uniform(-corrected_radius,corrected_radius)
	lon = center_lon + random.uniform(-corrected_radius, corrected_radius)

	# generate a date
	date = datetime.date(2024, 11, random.randint(1, 24))
	# generate a pest
	pest = random.choice(['Podredumbre blanca', 'Moho gris', 'Mosca minadora', 'Polilla de calabaza', 'Araña roja'])

	desc = pests.get(pest,'')

	# generate the input data
	input_data = {
		'Item': {
            'country': {'S': "Peru"},
            'region': {'S': region['name']},
            'date': {'S': date.isoformat()},
						'pest': {'S': pest},
						'description': {'S': desc}
        },
        'ConditionExpression': "attribute_not_exists(hashKey)" 
	}
	# generate the point
	generated_point = dynamodbgeo.GeoPoint(lat,lon)
	data.append((input_data, generated_point))# 


In [12]:
# put the data in the table
for input_data, generated_point in data:
	geoDataManager.put_Point(dynamodbgeo.PutPointInput(
        generated_point, # latitude then latitude longitude
         str( uuid.uuid4()), # Use this to ensure uniqueness of the hash/range pairs.
         input_data # pass the dict here
        ))

In [16]:
# save the data in a json format as pests.json
"""
	{
            'country':  "Peru",
            'region': "Acolla | Matahuasi",
            'date': 'isodate',
						'pest': "",
						'description': "",
						'lat': float,
						'lon': float
        },
"""

import json
data_json = []
for input_data, generated_point in data:
	data_json.append({
		'country':  "Peru",
        'region': input_data['Item']['region']['S'],
        'date': input_data['Item']['date']['S'],
		'pest': input_data['Item']['pest']['S'],
		'description': input_data['Item']['description']['S'],
		'lat': generated_point.latitude,
		'lon': generated_point.longitude
	})


pests_dict = {"pests":data_json}

with open('pests.json', 'w') as f:
	json.dump(pests_dict, f)
print

<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [11]:
# Querying 95 meter from the center point (36.879131, 10.243057)
QueryRadiusInput={
        "FilterExpression": "Country = :val1",
        "ExpressionAttributeValues": {
            ":val1": {"S": "Peru"},
        }
    }

query_reduis_result=geoDataManager.queryRadius(
    dynamodbgeo.QueryRadiusRequest(
        dynamodbgeo.GeoPoint(36.879131, 10.243057), # center point
        95, QueryRadiusInput, sort = True)) # diameter

query_reduis_result

[{'Capital': {'S': 'Lima'},
  'hashKey': {'N': '136'},
  'geoJson': {'S': '36.879163,10.24312'},
  'Country': {'S': 'Peru'},
  'rangeKey': {'S': '387df65c-452f-4dfe-8494-73c0cec9f48b'},
  'geohash': {'N': '1360872775087296659'},
  'year': {'S': '1000'}}]